In [17]:

import numpy as np
import pandas as pd
import os
import re
import csv
from pathlib import Path
import chardet


### This notebook is used to spliet dialog and text parts from BERT_annotations folder and save as two new files 'name'_text.txt and 'name_dialog.txt' in new folder '/text' and '/dialog'
* extract_text_dialog(directory,file) is used to deal with one file
* s_to_csv calls on extract_text_dialog(directory,file) and is used to deal with the whole folder. 
* example for use it : s_to_csv('/data/BERT_annotations')

In [18]:
# used to split diaolog and text from one file and return in two part

def extract_text_dialog(directory,file):
    
    file_name = file
    file_path = os.path.join(directory, file_name)

    try:
        with open(file_path, 'r') as file:
            text = file.read()
    
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' does not exist.")
        return
    except IOError as e:
        print(f"Error: Could not read the file '{file_path}'. Error: {str(e)}")
        return

    texts = []
    #speaker_headings = []
    dialogs = []

    # Regex patterns to match each section
    text_pattern = re.compile(r'text:\s+(.*?)(?=(speaker_heading|scene_heading|dialog|$))', re.DOTALL)
    #speaker_heading_pattern = re.compile(r'speaker_heading:\s+(.*?)(?=(text|scene_heading|dialog|$))', re.DOTALL)
    dialog_pattern = re.compile(r'dialog:\s+(.*?)(?=(text|scene_heading|speaker_heading|$))', re.DOTALL)

    # Function to clean and split text into lines
    def clean_and_split_text(match):
        return [line.strip() for line in match.strip().split('\n') if line.strip()]

    # Find all matches and append to respective lists
    for match in text_pattern.findall(text):
        texts.extend(clean_and_split_text(match[0]))

    #for match in speaker_heading_pattern.findall(text):
    #    speaker_headings.extend(clean_and_split_text(match[0]))

    for match in dialog_pattern.findall(text):
        dialogs.extend(clean_and_split_text(match[0]))
    
    

    # Determine the maximum length to ensure even columns
    max_length = max(len(texts), len(dialogs))
   
    # Pad the shorter lists with empty strings
    texts.extend([''] * (max_length - len(texts)))
    #speaker_headings.extend([''] * (max_length - len(speaker_headings)))
    dialogs.extend([''] * (max_length - len(dialogs)))
    #print(texts)
    #print(dialogs)
    return texts,dialogs

In [21]:

from pathlib import Path
def s_to_csv(dir):
    name = []
    text = []
    dialogs = []
    last_dir = os.path.basename(dir)
    path_new1 = dir + '/text'
    path_new1 = Path(path_new1)
    path_new1.mkdir(parents=True,exist_ok=True)
    path_new2 = dir + '/dialog'
    path_new2 = Path(path_new2)
    path_new2.mkdir(parents=True,exist_ok=True)
    try:
        files = os.listdir(dir)
        for file in files:
            if file.endswith('.txt'):
                movie = file.replace('.txt', '')
                name.append(movie)
                
                a, b = extract_text_dialog(dir,file)
                #a = a.replace('text:','')
                #b = b.replace('dialog','')
                text.append(' '.join(a))
                dialogs.append(' '.join(b))
        
                name1 = movie + '_text.txt'
                name2 = movie + '_dialog.txt'
                
                file1 = os.path.join(path_new1, name1)
                file2 = os.path.join(path_new2, name2)
                
                with open(file1, 'w') as txtfile:
                    txtfile.write('\n'.join(a))
                
                with open(file2, 'w') as txtfile:
                    txtfile.write('\n'.join(b))
    #--------------------------this part can used to save all dialog and text part to a .csv file connected by id(if is necessary)
    #    out = last_dir + '.csv'
    #    out_csv = os.path.join(dir, out)
    #    with open(out_csv, 'w', newline='') as csvfile:
    #        csv_writer = csv.writer(csvfile)
    #        csv_writer.writerow(['Name', 'Dialogs', 'Text'])
    #        for i in range(len(name)):
    #            csv_writer.writerow([name[i], dialogs[i], text[i]])
        
        #print("Movies:", name)
        #print("Texts:", text)
        #print("Dialogs:", dialogs)
    #----------------------------------------------------------------    
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [22]:

s_to_csv('/Users/xiaozhouye/Desktop/neuefische/Capstone-project/data/screenplay_data/data/BERT_annotations/BERT_annotations')